In [7]:
import pandas as pd
import re
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
import joblib

LEFT = 0
RIGHT = 1

articles = pd.read_csv('binary_bias.csv')

#randomizes rows
articles = articles.sample(frac=1)

stemmer = SnowballStemmer('english')
words = stopwords.words("english")

articles['cleaned'] = articles['article'].apply(lambda x: " ".join([stemmer.stem(i) for i in re.sub("[^a-zA-Z]", " ", x).split() if i not in words]).lower())

X_train, X_test, y_train, y_test = train_test_split(articles['cleaned'], articles['bias'], test_size=0.2)

pipeline = Pipeline([('vect', TfidfVectorizer(ngram_range=(1, 2), stop_words="english")),
                     ('chi',  SelectKBest(k=10000)),
                     ('clf', LinearSVC(C=1.0, penalty='l2', max_iter=6000, dual=False))])

model = pipeline.fit(X_train, y_train)

joblib.dump(model, 'demo_model.joblib')

vectorizer = model.named_steps['vect']
chi = model.named_steps['chi']
clf = model.named_steps['clf']

feature_names = vectorizer.get_feature_names()
feature_names = [feature_names[i] for i in chi.get_support(indices=True)]
feature_names = np.asarray(feature_names)

target_names = ['Left', 'Right']

print("\nAccuracy: " + str(round(model.score(X_test, y_test)*100, 2)) + "%\n")

print("\nTop 10 keywords for bias:\n")
top10 = np.argsort(clf.coef_[0])[-10:]
print("%s: %s" % (label, " ".join(feature_names[top10])))




Accuracy: 71.51%


Top 10 keywords for bias:

Right: support tax administr obama liber document presid illinoi arm obamacar
